# Simple GRU network with pretrained vectors for initialization

In [ ]:
import sys, os, re, csv, codecs, gc, numpy as np, pandas as pd
import tensorflow as tf
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Permute, GRU, Conv1D, LSTM, Embedding, Dropout, Activation, CuDNNLSTM, CuDNNGRU, concatenate, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, BatchNormalization, SpatialDropout1D, Dot
from keras.optimizers import Adam, RMSprop
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras_tqdm import TQDMNotebookCallback
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from functools import reduce
from keras.layers import Layer, PReLU, SpatialDropout1D
from keras import initializers
from sklearn.model_selection import cross_val_predict

from nltk.tokenize import word_tokenize, wordpunct_tokenize, TweetTokenizer, MWETokenizer, ToktokTokenizer
from nltk.corpus import stopwords

import unicodedata
from collections import Counter
import itertools

np.random.seed(786)

from Tokenizer import Tokenizer
from ZeroMaskedLayer import ZeroMaskedLayer
from AttentionLayer import AttentionLayer, AttentionWithContext, Attention

In [2]:
path = '../input/'
utility_path = '../utility/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE=f'{utility_path}glove.42B.300d.txt'
TRAIN_DATA_FILE=f'{path}train.csv'
TEST_DATA_FILE=f'{path}test.csv'

In [3]:
word_tokenize("I won't do this check!!!")

['I', 'wo', "n't", 'do', 'this', 'check', '!', '!', '!']

In [4]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [51]:
def unicodeToAscii(series):
    return series.apply(lambda s: unicodedata.normalize('NFKC', str(s)))


def multiple_replace(text, adict):
    rx = re.compile('|'.join(map(re.escape, adict)))

    def one_xlat(match):
        return adict[match.group(0)]

    return rx.sub(one_xlat, text)

STOP_WORDS = set(stopwords.words( 'english' ))
# Lowercase, trim, and remove non-letter characters
def normalizeString(series):
    series = unicodeToAscii(series)
    series = series.str.lower()
    series = series.str.replace(r"(\n){1,}", " ")
    series = series.str.replace(r"\'", "")
    series = series.str.replace(r"\-", "")
    series = series.str.replace(r"[^0-9a-zA-Z.,!?\"']+", " ")
    series = series.str.replace("([a-z0-9]{2,}\.){2,}[a-z]{2,}", " ") 
    #Replace URL's by url
    series = series.str.replace("\d", "")

    return series


In [50]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

print(train.shape, test.shape)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values

#Get validation folds
train['target_str'] = reduce(lambda x,y: x+y, [train[col].astype(str) for col in list_classes])
train['target_str'] = train['target_str'].replace('110101', '000000').replace('110110','000000')
cvlist1 = list(StratifiedKFold(n_splits=10, random_state=786).split(train, train['target_str'].astype('category')))
cvlist2 = list(StratifiedShuffleSplit(n_splits=5, test_size=0.05, random_state=786).split(train, train['target_str'].astype('category')))

(159571, 8) (153164, 2)


/home/mohsin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [52]:
for df in train, test:
    df["comment_text"] = normalizeString(df["comment_text"])

In [75]:
def strip_duplicates(df, thresh=10):
    inds = (df.comment_text.str.split(' ').str.len()/df.comment_text.str.split(' ').apply(lambda x: len(np.unique(x)))) >= thresh
    df.loc[inds, "comment_text"] = df.loc[inds].comment_text.str.split().apply(lambda x: ' '.join(x[:thresh]))
    return df

for df in train, test:
    df = strip_duplicates(df)

In [9]:
#pd.concat([train["comment_text"].astype(str), test["comment_text"].astype(str)]).reset_index(drop=True)[:len(train), :]

In [82]:
MAX_FEATURES = 150000
MAX_LEN = 200

tok = Tokenizer(max_features=MAX_FEATURES, max_len=MAX_LEN, tokenizer=word_tokenize)
X = tok.fit_transform(pd.concat([train["comment_text"].astype(str), test["comment_text"].astype(str)]))
X_train = X[:len(train), :]
X_test = X[len(train):, :]

print(X_train.shape, X_test.shape)

(159571, 200) (153164, 200)


In [83]:
tok.doc_freq.most_common(150000)[-100:]

[('meroaustin', 1),
 ('shamrockrockthrasheraustin', 1),
 ('mankindtafkag', 1),
 ('goldustchainz', 1),
 ('jarretthartvader', 1),
 ('johnsonhenry', 1),
 ('brownmark', 1),
 ('henryfaarooq', 1),
 ('skullno', 1),
 ('competekama', 1),
 ('mustafaaustin', 1),
 ('austinwinner', 1),
 ('vegaaustinfaarooqrock', 1),
 ('lovefaarooqchainzaustinvadertafkag', 1),
 ('mikhlin', 1),
 ('grafakos', 1),
 ('unrefernced', 1),
 ('hangulphaspa', 1),
 ('gravedancer', 1),
 ('cascadingly', 1),
 ('currentmonth', 1),
 ('misdisinformation', 1),
 ('goldfritter', 1),
 ('quickstep', 1),
 ('atisket', 1),
 ('atasket', 1),
 ('dowdy', 1),
 ('bieszczadzkie', 1),
 ('stivell', 1),
 ('kej', 1),
 ('talk.edits', 1),
 ('daaeefcbbebd', 1),
 ('rationalskeptic', 1),
 ('illequiped', 1),
 ('ivorytower', 1),
 ('condiser', 1),
 ('cuplex', 1),
 ('tungilik', 1),
 ('copyrightprotected', 1),
 ('libs.', 1),
 ('eams', 1),
 ('ritsos', 1),
 ('revertwarriors', 1),
 ('pagebanned', 1),
 ('sadass', 1),
 ('wikifortitude', 1),
 ('informationweek', 1),


In [84]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()


In [85]:
sp.Load("../utility/en.wiki.bpe.op200000.model")


True

In [86]:
sp.EncodeAsPieces("bitches.fuck")

['▁bitches', '.', 'fuck']

In [87]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open("../utility/en.wiki.bpe.op200000.d300.w2v.txt"))
len(embeddings_index)

187990

In [99]:
#embeddings_index
MAX_FEATURES = 122000
MAX_LEN = 200
tok = Tokenizer(max_features=MAX_FEATURES, max_len=MAX_LEN, tokenizer=sp.EncodeAsPieces)
X = tok.fit_transform(pd.concat([train["comment_text"].astype(str), test["comment_text"].astype(str)]))
X_train = X[:len(train), :]
X_test = X[len(train):, :]

print(X_train.shape, X_test.shape)

(159571, 200) (153164, 200)


In [100]:
len(tok.doc_freq)

122634

In [101]:
EMBED_SIZE = 300
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
oov_list =[]
def initialize_embeddings(filename, embeddings_index, tokenizer):
    #embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(filename))

    word_index = tokenizer.vocab_idx
    nb_words = min(MAX_FEATURES+1, len(word_index)+1)
    embedding_matrix = np.zeros((nb_words, EMBED_SIZE))
    for word, i in word_index.items():
        if i > MAX_FEATURES: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            
            #broken_words = sp.EncodeAsPieces(word)
            #broken_words()
            oov_list.append(word)
    return (embedding_matrix, oov_list)

In [102]:
tok.doc_freq.most_common(100000)[-100:]

[('salvia', 1),
 ('▁bagan', 1),
 ('bagan', 1),
 ('achov', 1),
 ('▁reposs', 1),
 ('▁reinsurance', 1),
 ('shoulder', 1),
 ('▁gwenn', 1),
 ('▁engen', 1),
 ('▁horsman', 1),
 ('enched', 1),
 ('▁renouf', 1),
 ('ursor', 1),
 ('▁merwin', 1),
 ('▁freist', 1),
 ('▁osip', 1),
 ('▁vertov', 1),
 ('▁bouy', 1),
 ('▁paleogene', 1),
 ('▁quins', 1),
 ('▁tannen', 1),
 ('▁fogh', 1),
 ('enhauer', 1),
 ('agro', 1),
 ('▁passerine', 1),
 ('▁vremya', 1),
 ('▁appia', 1),
 ('▁tennent', 1),
 ('▁encyclopa', 1),
 ('▁methil', 1),
 ('charlotte', 1),
 ('▁bronfman', 1),
 ('aghir', 1),
 ('▁thiy', 1),
 ('kki', 1),
 ('▁phonon', 1),
 ('abakh', 1),
 ('▁vram', 1),
 ('▁conover', 1),
 ('▁speechwriter', 1),
 ('asius', 1),
 ('▁seneschal', 1),
 ('▁photoplay', 1),
 ('▁popham', 1),
 ('▁marcom', 1),
 ('▁statu', 1),
 ('▁gendarme', 1),
 ('▁celestine', 1),
 ('▁hlinka', 1),
 ('▁plantarum', 1),
 ('▁wenzhou', 1),
 ('yuri', 1),
 ('rescent', 1),
 ('▁squal', 1),
 ('▁lepro', 1),
 ('clawed', 1),
 ('▁rasmus', 1),
 ('▁winther', 1),
 ('▁maturin',

In [103]:
embedding_matrix, oov_list = initialize_embeddings(EMBEDDING_FILE, embeddings_index, tok)
print(embedding_matrix.shape)
print(np.mean(embedding_matrix), np.std(embedding_matrix))

(122001, 300)
0.0007162527987300298 0.2728021219348521


In [104]:
print(len(oov_list))
oov_list

1629


['▁reall',
 'utzer',
 'cile',
 '▁advertis',
 'incha',
 '▁jizz',
 '▁pagen',
 'acuse',
 'onsored',
 'amatory',
 '▁inadvert',
 '▁clich',
 'ecially',
 '▁erics',
 '▁attem',
 '▁answ',
 'ungeons',
 '▁deleter',
 '▁encycl',
 '▁makedon',
 '▁antide',
 '▁behav',
 '▁excer',
 'trypt',
 '▁toky',
 'irected',
 '▁embarr',
 '▁mahaw',
 '▁undoub',
 '▁acknow',
 'reputable',
 'schnitt',
 '▁poved',
 'olnick',
 '▁eshte',
 '▁welcom',
 '▁criter',
 '▁irond',
 '▁copyr',
 '▁metaw',
 'atever',
 'leves',
 '▁scientif',
 '▁appre',
 'adays',
 '▁proport',
 '▁confir',
 '▁inconsist',
 '▁pejor',
 'eday',
 '▁euthym',
 'urday',
 'ypedia',
 'piff',
 'oburg',
 'illaz',
 '▁ethiop',
 'mosin',
 '▁einst',
 'itorul',
 'chihu',
 '▁schnitz',
 'viously',
 '▁tomor',
 '▁zapat',
 '▁sevend',
 'ruction',
 '▁recip',
 'ignatus',
 '▁refere',
 '▁achiev',
 'amilton',
 '▁suce',
 '▁compre',
 'ullivan',
 '▁joaqu',
 'tirol',
 'namese',
 '▁indones',
 '▁resemb',
 'ailando',
 '▁yusufali',
 'icance',
 '▁alask',
 'otre',
 '▁errone',
 '▁leban',
 '▁outrig'

In [146]:
from sklearn.base import BaseEstimator, ClassifierMixin
class GRUClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, gru_dim=150, dense_dim=256, batch_size=128, epochs=2, bidirectional=False, 
                 pool_type='all', initial_weights=None, optimizer='adam' ,verbose=1, out_dim=6, callbacks=None,
                spatial_drop=0.0, dropout=0.0, mask_zero=True, 
                gru_kernel_regularization = 0.0,
                gru_recurrent_regularization = 0.0,
                gru_bias_regularization = 0.0,
                embeddings_regularization = 0.0,
                ):
        
        self.gru_dim = gru_dim
        self.dense_dim = dense_dim
        self.batch_size = batch_size
        self.epochs= epochs
        self.bidirectional = bidirectional
        self.pool_type = pool_type
        self.initial_weights = initial_weights
        self.verbose = verbose
        self.callbacks = callbacks
        self.optimizer = optimizer
        self.out_dim = out_dim
        self.spatial_drop = spatial_drop
        self.dropout = dropout
        self.mask_zero = mask_zero
        self.gru_kernel_regularization = gru_kernel_regularization
        self.gru_recurrent_regularization = gru_recurrent_regularization
        self.gru_bias_regularization = gru_bias_regularization
        self.embeddings_regularization = embeddings_regularization
        
    def _build_model(self):
        inp = Input(shape=(MAX_LEN,))
        emb = Embedding(MAX_FEATURES+1, 
                        EMBED_SIZE,
                        weights=[self.initial_weights],
                        mask_zero=self.mask_zero,
                        #embeddings_regularizer=regularizers.l2(self.embeddings_regularization),
                        trainable=False)(inp)

        if self.mask_zero:
            emb = ZeroMaskedLayer()(emb)
         
        #emb2 = Dot(axes=1)([emb, emb])
        #print(emb.shape)
        emb2 = SpatialDropout1D(self.spatial_drop)(emb)
        if self.bidirectional:
            enc = Bidirectional(CuDNNGRU(int(self.gru_dim), return_sequences=True, return_state=True, stateful=False,
                                     kernel_initializer="he_uniform"))(emb2)
            x = enc[0]
            state = enc[1]
        else:
            x, state = CuDNNGRU(int(self.gru_dim), return_sequences=True, return_state=True,
                            kernel_regularizer=regularizers.l2(self.gru_kernel_regularization),
                            recurrent_regularizer=regularizers.l2(self.gru_recurrent_regularization),
                            bias_regularizer=regularizers.l2(self.gru_bias_regularization)
                               )(emb2)
            #x = SpatialDropout1D(0.5)(x)
        
        if self.pool_type == 'avg':
            x = GlobalAveragePooling1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'max':
            x = GlobalMaxPool1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'attn':
            x = AttentionLayer(MAX_LEN)(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'all':
            #x1 = GlobalAveragePooling1D()(emb2)
            x2 = GlobalAveragePooling1D()(x)
            #x3 = GlobalAveragePooling1D()(emb)
            x4 = GlobalMaxPool1D()(x)
            x5 = AttentionLayer(MAX_LEN)(x)
            x = concatenate([ x4, x5, state])
    
        #x = Dropout(self.dropout)(x)
        #x = Dense(self.dense_dim, kernel_initializer="he_uniform")(x)
        #x = PReLU()(x)
        
        #x = Dense(600)(x)
        #x = PReLU()(x)

        out = Dense(self.out_dim, activation="sigmoid")(x)
        if self.optimizer == 'adam':
            opt = Adam(lr=0.001, decay=0.0)
        elif self.optimizer == 'rmsprop':
            opt = RMSprop(clipnorm=1.0)
        model = Model(inputs=inp, outputs=out)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return model
    
    def fit(self, X, y):
        self.model = self._build_model()
        
        if self.callbacks:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                       callbacks=self.callbacks,
                       shuffle=True)
        else:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                       shuffle=True)
        return self
    
    def predict(self, X, y=None):
        if self.model:
            y_hat = self.model.predict(X, batch_size=1024)
        else:
            raise ValueError("Model not fit yet")
        return y_hat

In [144]:
def lr_decay(epoch):
    if epoch == 0:
        return 0.0016
    if epoch == 1:
        return 0.00024
    if epoch == 2:
        return 0.001
    if epoch == 3:
        return 0.00001


def shuffle_crossvalidator(model, cvlist, X, y, lr_decay):
    y_trues = []
    y_preds = []
    scores = []
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)
        print("ROC AUC for this fold is ", score)
        y_trues.append(y_val)
        y_preds.append(y_pred)
        K.clear_session()
        gc.collect()
        #break
    y_trues = np.concatenate(y_trues)
    y_preds = np.concatenate(y_preds)
    score = roc_auc_score(y_trues, y_preds)
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, scores

def outoffold_crossvalidator(model_params, cvlist, X, y, lr_decay):
    y_preds = np.zeros(y.shape)
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
        
        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        print("ROC AUC for this fold is ", roc_auc_score(y_val, y_pred))
        y_preds[val_idx] = y_pred
        K.clear_session()
        break
    score = roc_auc_score(y, y_preds)
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, score


In [147]:
import tensorflow as tf
def lr_decay(epoch):
    if epoch == 0:
        return 0.001
    if epoch == 1:
        return 0.001
    if epoch == 2:
        return 0.001
    if epoch == 3:
        return 0.001
    if epoch == 4:
        return 0.001
    if epoch == 5:
        return 0.001
    return 0.001

K.clear_session()
#config = tf.ConfigProto(
#        device_count = {'GPU': 1}
#    )
#sess = tf.Session(config=config)
model = GRUClassifier(gru_dim=400, dense_dim=300, initial_weights=embedding_matrix, bidirectional=False,
                    batch_size=256, epochs=4, optimizer='adam', pool_type='all', dropout=0.2, spatial_drop=0.0, mask_zero=False)

y_preds, y_trues, _ = shuffle_crossvalidator(model, cvlist2, X_train, y, lr_decay)

Epoch 1/4
151552/151592 [============================>.] - ETA: 0s - loss: 0.0674 - acc: 0.9772
 ROC-AUC - epoch: 1 - score: 0.979516 

151592/151592 [==============================] - 39s 260us/step - loss: 0.0674 - acc: 0.9772
Epoch 2/4
 82432/151592 [===============>..............] - ETA: 18s - loss: 0.0462 - acc: 0.9826

KeyboardInterrupt: 

In [26]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


def uniform_int(name, lower, upper):
    # `quniform` returns:
    # round(uniform(low, high) / q) * q
    return hp.quniform(name, lower, upper, q=1)

def loguniform_int(name, lower, upper):
    # Do not forget to make a logarithm for the
    # lower and upper bounds.
    return hp.qloguniform(name, np.log(lower), np.log(upper), q=1)

parameter_space = {
    'gru_dim': uniform_int('gru_dim', 100, 600),
    'dense_dim': uniform_int('dense_dim', 100, 1200),
    'lr1': hp.uniform('lr1', 0.0001, 0.002),
    'lr2': hp.uniform('lr2', 0.0001, 0.002),
    'spatial_drop': hp.uniform('spatial_drop', 0, 0.5),
    'dropout': hp.uniform('dropout', 0, 0.5),
    'batch_size': loguniform_int('batch_size', 16, 1028),
    'epochs': hp.choice('epochs', [2,3])
    #'mask_zero': hp.choice('mask_zero', [True, False]),
    #'optimizer': hp.choice('optimizer', ['adam', 'rmsprop']),
    #'pool_type': hp.choice('pool_type', ['avg', 'max', 'attn', 'all']),
    #'bidirectional': hp.choice('bidirectional', [True, False]),
    #'gru_kernel_reg': hp.loguniform('gru_kernel_reg', np.log(1e-10), np.log(1e-4)),
    #'gru_recc_reg': hp.loguniform('gru_recc_reg', np.log(1e-10), np.log(1e-4)),
    #'gru_bias_reg': hp.loguniform('gru_bias_reg', np.log(1e-10), np.log(1e-4)),
    #'embeddings_reg': hp.loguniform('embeddings_reg', 1e-8, 1e-4)
}


def objective(parameter_space):
    
    def lr_decay(epoch):
        if epoch == 0:
            return parameter_space['lr1']
        if epoch == 1:
            return parameter_space['lr2']
        return parameter_space['lr2']
    
    model = GRUClassifier(initial_weights=embedding_matrix, bidirectional=False,
                          gru_dim = int(parameter_space['gru_dim']),
                          dense_dim = int(parameter_space['dense_dim']),
                          mask_zero = False,
                          pool_type = 'all',
                          batch_size= int(parameter_space['batch_size']), 
                          epochs=int(parameter_space['epochs']), 
                          optimizer='adam',
                          dropout=parameter_space['dropout'],
                          spatial_drop=parameter_space['spatial_drop'],
                          gru_kernel_regularization = 0,
                          gru_recurrent_regularization = 0,
                          gru_bias_regularization = 0,
                          #embeddings_regularization = parameter_space["embeddings_reg"],
                          )

    y_preds, y_trues, scores = shuffle_crossvalidator(model, cvlist2, X_train, y, lr_decay)    
    score = roc_auc_score(y_trues, y_preds)
    print("Score for parameters {} is {}".format(parameter_space, score))
    #return score
    return {
        'loss': -1* score,
        'status': STATUS_OK,
        'other_Stuff': {'scores': scores, 'variance': np.std(scores)},
        }

trials = Trials()

best = fmin(objective,
    space=parameter_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
           )

print(best)

Epoch 1/2
151351/151592 [============================>.] - ETA: 0s - loss: 0.0848 - acc: 0.9733
 ROC-AUC - epoch: 1 - score: 0.977045 

151592/151592 [==============================] - 24s 157us/step - loss: 0.0848 - acc: 0.9733
Epoch 2/2
151351/151592 [============================>.] - ETA: 0s - loss: 0.0464 - acc: 0.9824
 ROC-AUC - epoch: 2 - score: 0.983278 

151592/151592 [==============================] - 24s 158us/step - loss: 0.0464 - acc: 0.9824
ROC AUC for this fold is  0.9832782173298816
Epoch 1/2
151351/151592 [============================>.] - ETA: 0s - loss: 0.0841 - acc: 0.9732
 ROC-AUC - epoch: 1 - score: 0.981172 

151592/151592 [==============================] - 24s 156us/step - loss: 0.0841 - acc: 0.9732
Epoch 2/2
151351/151592 [============================>.] - ETA: 0s - loss: 0.0457 - acc: 0.9825
 ROC-AUC - epoch: 2 - score: 0.987718 

151592/151592 [==============================] - 24s 156us/step - loss: 0.0457 - acc: 0.9825
ROC AUC for this fold is  0.98771812283

Epoch 3/3
151544/151592 [============================>.] - ETA: 0s - loss: 0.0342 - acc: 0.9860
 ROC-AUC - epoch: 3 - score: 0.989000 

151592/151592 [==============================] - 108s 710us/step - loss: 0.0342 - acc: 0.9860
ROC AUC for this fold is  0.9889995102064396
Epoch 1/3
151544/151592 [============================>.] - ETA: 0s - loss: 0.2247 - acc: 0.9042
 ROC-AUC - epoch: 1 - score: 0.942581 

151592/151592 [==============================] - 109s 716us/step - loss: 0.2249 - acc: 0.9041
Epoch 2/3
151468/151592 [============================>.] - ETA: 0s - loss: 0.6564 - acc: 0.7099
 ROC-AUC - epoch: 2 - score: 0.942581 

151592/151592 [==============================] - 108s 713us/step - loss: 0.6564 - acc: 0.7099
Epoch 3/3
151544/151592 [============================>.] - ETA: 0s - loss: 0.6566 - acc: 0.7103
 ROC-AUC - epoch: 3 - score: 0.942581 

151592/151592 [==============================] - 107s 707us/step - loss: 0.6566 - acc: 0.7103
ROC AUC for this fold is  0.9439664

Epoch 1/3
151548/151592 [============================>.] - ETA: 0s - loss: 0.0515 - acc: 0.9812
 ROC-AUC - epoch: 1 - score: 0.985771 

151592/151592 [==============================] - 141s 929us/step - loss: 0.0515 - acc: 0.9812
Epoch 2/3
151548/151592 [============================>.] - ETA: 0s - loss: 0.0407 - acc: 0.9841
 ROC-AUC - epoch: 2 - score: 0.981815 

151592/151592 [==============================] - 141s 929us/step - loss: 0.0407 - acc: 0.9841
Epoch 3/3
151548/151592 [============================>.] - ETA: 0s - loss: 0.0319 - acc: 0.9869
 ROC-AUC - epoch: 3 - score: 0.987973 

151592/151592 [==============================] - 143s 941us/step - loss: 0.0319 - acc: 0.9869
ROC AUC for this fold is  0.9879725475584151
Overall score on 10 fold CV is 0.9857641970818456
Score for parameters {'batch_size': 73.0, 'dense_dim': 738.0, 'dropout': 0.1485005581562424, 'epochs': 3, 'gru_dim': 526.0, 'lr1': 0.001554543004233458, 'lr2': 0.0008439679745947272, 'spatial_drop': 0.21831678303864

Epoch 3/3
151410/151592 [============================>.] - ETA: 0s - loss: 0.0369 - acc: 0.9850
 ROC-AUC - epoch: 3 - score: 0.988272 

151592/151592 [==============================] - 65s 431us/step - loss: 0.0368 - acc: 0.9850
ROC AUC for this fold is  0.9882722272522608
Epoch 1/3
151410/151592 [============================>.] - ETA: 0s - loss: 0.0616 - acc: 0.9785
 ROC-AUC - epoch: 1 - score: 0.986545 

151592/151592 [==============================] - 65s 431us/step - loss: 0.0616 - acc: 0.9785
Epoch 2/3
151410/151592 [============================>.] - ETA: 0s - loss: 0.0405 - acc: 0.9840
 ROC-AUC - epoch: 2 - score: 0.989540 

151592/151592 [==============================] - 65s 431us/step - loss: 0.0405 - acc: 0.9840
Epoch 3/3
151410/151592 [============================>.] - ETA: 0s - loss: 0.0372 - acc: 0.9850
 ROC-AUC - epoch: 3 - score: 0.990786 

151592/151592 [==============================] - 66s 433us/step - loss: 0.0372 - acc: 0.9850
ROC AUC for this fold is  0.99078636312

151145/151592 [============================>.] - ETA: 0s - loss: 0.0783 - acc: 0.9737
 ROC-AUC - epoch: 1 - score: 0.981483 

151592/151592 [==============================] - 24s 162us/step - loss: 0.0783 - acc: 0.9737
Epoch 2/3
151145/151592 [============================>.] - ETA: 0s - loss: 0.0425 - acc: 0.9836
 ROC-AUC - epoch: 2 - score: 0.984895 

151592/151592 [==============================] - 24s 161us/step - loss: 0.0424 - acc: 0.9836
Epoch 3/3
151145/151592 [============================>.] - ETA: 0s - loss: 0.0365 - acc: 0.9854
 ROC-AUC - epoch: 3 - score: 0.986412 

151592/151592 [==============================] - 25s 162us/step - loss: 0.0365 - acc: 0.9854
ROC AUC for this fold is  0.9864123534075177
Epoch 1/3
151145/151592 [============================>.] - ETA: 0s - loss: 0.0761 - acc: 0.9750
 ROC-AUC - epoch: 1 - score: 0.977796 

151592/151592 [==============================] - 25s 162us/step - loss: 0.0760 - acc: 0.9750
Epoch 2/3
151145/151592 [========================

...

Epoch 1/2
151590/151592 [============================>.] - ETA: 0s - loss: 0.0708 - acc: 0.9771
 ROC-AUC - epoch: 1 - score: 0.978956 

151592/151592 [==============================] - 62s 409us/step - loss: 0.0708 - acc: 0.9771
Epoch 2/2
151590/151592 [============================>.] - ETA: 0s - loss: 0.0454 - acc: 0.9825
 ROC-AUC - epoch: 2 - score: 0.989675 

151592/151592 [==============================] - 62s 411us/step - loss: 0.0454 - acc: 0.9825
ROC AUC for this fold is  0.9896749609237464
Epoch 1/2
151590/151592 [============================>.] - ETA: 0s - loss: 0.0721 - acc: 0.9759
 ROC-AUC - epoch: 1 - score: 0.980768 

151592/151592 [==============================] - 62s 407us/step - loss: 0.0721 - acc: 0.9759
Epoch 2/2
151590/151592 [============================>.] - ETA: 0s - loss: 0.0450 - acc: 0.9826
 ROC-AUC - epoch: 2 - score: 0.989511 

151592/151592 [==============================] - 62s 406us/step - loss: 0.0450 - acc: 0.9826
ROC AUC for this fold is  0.98951143716

Epoch 1/2
151493/151592 [============================>.] - ETA: 0s - loss: 0.0627 - acc: 0.9784
 ROC-AUC - epoch: 1 - score: 0.981351 

151592/151592 [==============================] - 48s 319us/step - loss: 0.0626 - acc: 0.9784
Epoch 2/2
151493/151592 [============================>.] - ETA: 0s - loss: 0.0428 - acc: 0.9834
 ROC-AUC - epoch: 2 - score: 0.983781 

151592/151592 [==============================] - 48s 318us/step - loss: 0.0428 - acc: 0.9834
ROC AUC for this fold is  0.983780538189257
Epoch 1/2
151493/151592 [============================>.] - ETA: 0s - loss: 0.0640 - acc: 0.9781
 ROC-AUC - epoch: 1 - score: 0.984498 

151592/151592 [==============================] - 48s 318us/step - loss: 0.0640 - acc: 0.9781
Epoch 2/2
151493/151592 [============================>.] - ETA: 0s - loss: 0.0431 - acc: 0.9834
 ROC-AUC - epoch: 2 - score: 0.987275 

151592/151592 [==============================] - 48s 319us/step - loss: 0.0431 - acc: 0.9834
ROC AUC for this fold is  0.987275203683

Epoch 3/3
151536/151592 [============================>.] - ETA: 0s - loss: 0.0293 - acc: 0.9879
 ROC-AUC - epoch: 3 - score: 0.990485 

151592/151592 [==============================] - 91s 602us/step - loss: 0.0293 - acc: 0.9879
ROC AUC for this fold is  0.9904847884147706
Epoch 1/3
151536/151592 [============================>.] - ETA: 0s - loss: 0.0520 - acc: 0.9808
 ROC-AUC - epoch: 1 - score: 0.987049 

151592/151592 [==============================] - 91s 601us/step - loss: 0.0520 - acc: 0.9808
Epoch 2/3
151536/151592 [============================>.] - ETA: 0s - loss: 0.0330 - acc: 0.9866
 ROC-AUC - epoch: 2 - score: 0.988611 

151592/151592 [==============================] - 91s 601us/step - loss: 0.0330 - acc: 0.9866
Epoch 3/3
151536/151592 [============================>.] - ETA: 0s - loss: 0.0294 - acc: 0.9879
 ROC-AUC - epoch: 3 - score: 0.988610 

151592/151592 [==============================] - 91s 600us/step - loss: 0.0294 - acc: 0.9879
ROC AUC for this fold is  0.98860994154

151583/151592 [============================>.] - ETA: 0s - loss: 0.0318 - acc: 0.9871
 ROC-AUC - epoch: 3 - score: 0.988899 

151592/151592 [==============================] - 169s 1ms/step - loss: 0.0318 - acc: 0.9871
ROC AUC for this fold is  0.9888987769588716
Epoch 1/3
151583/151592 [============================>.] - ETA: 0s - loss: 0.0526 - acc: 0.9809
 ROC-AUC - epoch: 1 - score: 0.984158 

151592/151592 [==============================] - 169s 1ms/step - loss: 0.0526 - acc: 0.9809
Epoch 2/3
151583/151592 [============================>.] - ETA: 0s - loss: 0.0349 - acc: 0.9859
 ROC-AUC - epoch: 2 - score: 0.987080 

151592/151592 [==============================] - 169s 1ms/step - loss: 0.0349 - acc: 0.9859
Epoch 3/3
151583/151592 [============================>.] - ETA: 0s - loss: 0.0305 - acc: 0.9875
 ROC-AUC - epoch: 3 - score: 0.986084 

151592/151592 [==============================] - 169s 1ms/step - loss: 0.0305 - acc: 0.9875
ROC AUC for this fold is  0.9860844607387799
Epoch 1/

151580/151592 [============================>.] - ETA: 0s - loss: 0.0360 - acc: 0.9856
 ROC-AUC - epoch: 3 - score: 0.989692 

151592/151592 [==============================] - 241s 2ms/step - loss: 0.0360 - acc: 0.9856
ROC AUC for this fold is  0.9896923000851413
Epoch 1/3
151580/151592 [============================>.] - ETA: 0s - loss: 0.0548 - acc: 0.9803
 ROC-AUC - epoch: 1 - score: 0.983941 

151592/151592 [==============================] - 239s 2ms/step - loss: 0.0548 - acc: 0.9803
Epoch 2/3
151580/151592 [============================>.] - ETA: 0s - loss: 0.0392 - acc: 0.9844
 ROC-AUC - epoch: 2 - score: 0.986203 

151592/151592 [==============================] - 236s 2ms/step - loss: 0.0392 - acc: 0.9844
Epoch 3/3
151580/151592 [============================>.] - ETA: 0s - loss: 0.0357 - acc: 0.9856
 ROC-AUC - epoch: 3 - score: 0.987367 

151592/151592 [==============================] - 239s 2ms/step - loss: 0.0357 - acc: 0.9856
ROC AUC for this fold is  0.9873673698158695
Overall 

In [36]:
import pickle
with open("../utility/BPEtrials.pkl", "wb") as f:
    pickle.dump(trials, f)

In [39]:
top_trials = sorted(trials.trials, key=lambda x: x['result']['loss'])[:5]
parameter_list = [t['misc']['vals'] for t in top_trials]
parameter_list

[{'batch_size': [24.0],
  'dense_dim': [332.0],
  'dropout': [0.18328351455634154],
  'epochs': [0],
  'gru_dim': [497.0],
  'lr1': [0.0005567624253149127],
  'lr2': [0.00010341656902972877],
  'spatial_drop': [0.183030393101664]},
 {'batch_size': [20.0],
  'dense_dim': [409.0],
  'dropout': [0.024042667906696014],
  'epochs': [0],
  'gru_dim': [290.0],
  'lr1': [0.0007258833783314673],
  'lr2': [0.00020874639127085808],
  'spatial_drop': [0.22389350207590156]},
 {'batch_size': [44.0],
  'dense_dim': [369.0],
  'dropout': [0.18849809847787988],
  'epochs': [0],
  'gru_dim': [366.0],
  'lr1': [0.0009038526129043502],
  'lr2': [0.00012278735241880124],
  'spatial_drop': [0.012952989044521579]},
 {'batch_size': [40.0],
  'dense_dim': [116.0],
  'dropout': [0.19141611141960815],
  'epochs': [0],
  'gru_dim': [379.0],
  'lr1': [0.0011126931595417252],
  'lr2': [0.00012023233921716057],
  'spatial_drop': [0.14532748732976142]},
 {'batch_size': [42.0],
  'dense_dim': [599.0],
  'dropout': [0.

In [40]:
import gc 
gc.collect()
K.clear_session()

In [140]:
#Pick top 10 parameter settings, Bag models for those settings
#Try linear blending on those settings
NUM_BAGS = 5
cvlist3 = list(StratifiedShuffleSplit(n_splits=NUM_BAGS, test_size=0.025, random_state=786).split(y, train["target_str"]))


def shuffle_train_predict(model, cvlist, X, y, X_test, lr_decay):
    y_trues = []
    y_preds = []
    y_test_preds = []
    scores = []
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)
        print("ROC AUC for this fold is ", score)
        y_trues.append(y_val)
        y_preds.append(y_pred)
        y_test_preds.append(y_test_pred)
        K.clear_session()
        gc.collect()
        #break
    y_trues = np.concatenate(y_trues)
    y_preds = np.concatenate(y_preds)
    y_test_preds = np.mean(y_test_preds, axis=0)
    print("Shape of test _preds is ", y_test_preds.shape)
    print("Means of val and test preds are {} and {}".format(np.mean(y_preds, axis=1), np.mean(y_test_preds, axis=1)))
    score = roc_auc_score(y_trues, y_preds)
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, y_test_preds

def oof_train_predict(model, cvlist, X, y, X_test, lr_decay):
    #y_trues = []
    y_test_preds = []
    scores = []
    y_preds = np.zeros(y.shape)
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)
        print("ROC AUC for this fold is ", score)
        #y_trues.append(y_val)
        y_preds[val_index, :] = y_pred
        y_test_preds.append(y_test_pred)
        K.clear_session()
        gc.collect()
        #break
    #y_trues = np.concatenate(y_trues)
    #y_preds = np.concatenate(y_preds)
    y_test_preds = np.mean(y_test_preds, axis=0)
    print("Shape of test _preds is ", y_test_preds.shape)
    print("Means of val and test preds are {} and {}".format(np.mean(y_preds, axis=0), np.mean(y_test_preds, axis=0)))
    score = roc_auc_score(y, y_preds)
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_test_preds

def train_predict(parameter_space):
    
    def lr_decay(epoch):
        if epoch == 0:
            return parameter_space['lr1'][0]
        if epoch == 1:
            return parameter_space['lr2'][0]
    
    model = GRUClassifier(initial_weights=embedding_matrix, bidirectional=False,
                          gru_dim = int(parameter_space['gru_dim'][0]),
                          dense_dim = int(parameter_space['dense_dim'][0]),
                          mask_zero = False,
                          pool_type = 'all',
                          batch_size= int(parameter_space['batch_size'][0]), 
                          epochs=2, 
                          optimizer='adam',
                          dropout=parameter_space['dropout'][0],
                          spatial_drop=parameter_space['spatial_drop'][0],
                          gru_kernel_regularization = 0,
                          gru_recurrent_regularization = 0,
                          gru_bias_regularization = 0,
                          #embeddings_regularization = parameter_space["embeddings_reg"],
                          )

    y_preds, y_trues, y_test_preds = shuffle_train_predict(model, cvlist3, X_train, y, X_test, lr_decay) 
    #y_preds, y_test_preds = oof_train_predict(model, cvlist2, X_train, y, X_test, lr_decay)
    return y_preds, y_trues, y_test_preds

#####
y_preds_all = []
y_trues_all = []
y_test_preds_all = []
for params in parameter_list:
    y_preds, y_test_preds = train_predict(params)
    y_preds_all.append(y_preds)
    y_trues_all.append(y_trues)
    y_test_preds_all.append(y_test_preds)


Epoch 1/2
155544/155581 [============================>.] - ETA: 0s - loss: 0.0556 - acc: 0.9800
 ROC-AUC - epoch: 1 - score: 0.987766 

155581/155581 [==============================] - 181s 1ms/step - loss: 0.0556 - acc: 0.9800
Epoch 2/2
155544/155581 [============================>.] - ETA: 0s - loss: 0.0418 - acc: 0.9837
 ROC-AUC - epoch: 2 - score: 0.990567 

155581/155581 [==============================] - 181s 1ms/step - loss: 0.0418 - acc: 0.9837
ROC AUC for this fold is  0.9905674556186123
Epoch 1/2
155544/155581 [============================>.] - ETA: 0s - loss: 0.0555 - acc: 0.9799
 ROC-AUC - epoch: 1 - score: 0.985087 

155581/155581 [==============================] - 181s 1ms/step - loss: 0.0555 - acc: 0.9799
Epoch 2/2
155544/155581 [============================>.] - ETA: 0s - loss: 0.0419 - acc: 0.9838
 ROC-AUC - epoch: 2 - score: 0.986711 

155581/155581 [==============================] - 180s 1ms/step - loss: 0.0420 - acc: 0.9837
ROC AUC for this fold is  0.986711226368188

ValueError: too many values to unpack (expected 2)

In [ ]:
###### Check corelation between different predictions
#np.corrcoef(y_preds_all, axis=0)

In [46]:
#Try different stacking approaches
from scipy.stats import gmean, hmean

preds_mean = gmean(y_preds_all, axis=0)
print(roc_auc_score(y, preds_mean))
test_preds_mean = gmean(y_test_preds_all, axis=0)


0.5256346108741795


/home/mohsin/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:305: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))


In [47]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_preds_mean
sample_submission.to_csv('../input/gru_spemb_5bags_submission.csv', index=False)